<a href="https://colab.research.google.com/github/IT-17005/Advanced-Cryptography/blob/main/PRNG_Algo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from typing import Optional, List
import time
import struct

MASK64 = (1 << 64) - 1

def _rotl(x, k):
    return ((x << k) & MASK64) | (x >> (64 - k))

def _splitmix64(seed):
    # Deterministic 64-bit splitmix step used for seeding/initialization
    z = (seed + 0x9e3779b97f4a7c15) & MASK64
    z = (z ^ (z >> 30)) * 0xbf58476d1ce4e5b9 & MASK64
    z = (z ^ (z >> 27)) * 0x94d049bb133111eb & MASK64
    return (z ^ (z >> 31)) & MASK64

class HybridPRNG:
    """
    A hybrid PRNG combining:
      - xorshift-style state transitions (3x 64-bit words)
      - a multiplicative/rotational output and a SplitMix-like finalizer
    Design goals: good mixing, fast, small state (3x64=192 bits), reproducible.
    """
    def __init__(self, seed: Optional[int] = None):
        if seed is None:
            seed = int(time.time() * 1000) ^ id(self)
        self._seed = seed & MASK64
        # initialize three 64-bit state words using splitmix64
        s0 = _splitmix64(self._seed)
        s1 = _splitmix64(s0)
        s2 = _splitmix64(s1)
        # avoid all-zero state
        if s0 == 0 and s1 == 0 and s2 == 0:
            s0 = 0x9e3779b97f4a7c15
            s1 = 0x243f6a8885a308d3
            s2 = 0x13198a2e03707344
        self.state = [s0, s1, s2]

    def reseed(self, seed: int):
        self.__init__(seed)

    def _next_core(self):
        # small xorshift/xoshiro-like core update
        s0, s1, s2 = self.state
        result = (s0 + s1 + s2) & MASK64
        # state update (inspired by xoshiro / xorshift families)
        s2 ^= s1
        s0 = _rotl(s0, 49) ^ s2 ^ ((s2 << 21) & MASK64)
        s1 = _rotl(s1, 28) ^ s0
        # write back
        self.state[0], self.state[1], self.state[2] = s0 & MASK64, s1 & MASK64, s2 & MASK64
        return result & MASK64

    def _finalize(self, x: int) -> int:
        # a splitmix-like finalizer to avalanche bits (not cryptographic)
        z = (x + 0x9e3779b97f4a7c15) & MASK64
        z = (z ^ (z >> 30)) * 0xbf58476d1ce4e5b9 & MASK64
        z = (z ^ (z >> 27)) * 0x94d049bb133111eb & MASK64
        z = z ^ (z >> 31)
        return z & MASK64

    def next_uint64(self) -> int:
        """Return next 64-bit unsigned integer"""
        x = self._next_core()
        return self._finalize(x)

    def random(self) -> float:
        """Return floating point in [0, 1) with 53-bit precision"""
        # use top 53 bits of next_uint64 to create double precision fraction
        u = self.next_uint64() >> 11  # keep top 53 bits
        return u / float(1 << 53)

    def randint(self, a: int, b: int) -> int:
        """Return integer uniformly in [a, b] inclusive"""
        if a > b:
            raise ValueError("a must be <= b")
        span = b - a + 1
        # rejection sampling to avoid bias
        while True:
            r = self.next_uint64()
            if span & (span - 1) == 0:
                # power of two optimization
                return a + (r & (span - 1))
            limit = ((1 << 64) // span) * span
            if r < limit:
                return a + (r % span)

    def randbytes(self, n: int) -> bytes:
        """Return n random bytes"""
        out = bytearray()
        while n > 0:
            r = self.next_uint64()
            chunk = r.to_bytes(8, 'little')
            take = min(n, 8)
            out += chunk[:take]
            n -= take
        return bytes(out)

    def shuffle(self, lst: List):
        """In-place Fisher-Yates shuffle using this PRNG"""
        for i in range(len(lst) - 1, 0, -1):
            j = self.randint(0, i)
            lst[i], lst[j] = lst[j], lst[i]

    def jump(self):
        """Advance internal state by a deterministic polynomial step (not full-period proof)"""
        # simple jump: perform several core steps to step ahead
        for _ in range(16):
            self._next_core()

    def get_state(self):
        return tuple(self.state)

    def set_state(self, s0: int, s1: int, s2: int):
        self.state = [s0 & MASK64, s1 & MASK64, s2 & MASK64]

if __name__ == "__main__":
    # small demo
    rng = HybridPRNG(123456789)
    print("first 5 uint64:", [rng.next_uint64() for _ in range(5)])
    rng2 = HybridPRNG(123456789)
    print("first 5 floats:", [rng2.random() for _ in range(5)])


first 5 uint64: [15249437416913338006, 16325474126650558970, 12644328681351958598, 14040183006591580526, 16808707429036105546]
first 5 floats: [0.8266736588299589, 0.8850057257485214, 0.6854504312971283, 0.7611198458920326, 0.9112018555617091]
